In [1]:
import pandas as pd

In [2]:
import anvil.server

anvil.server.connect("client_KYOM4YFDIE4TMEO3UAOTEPRC-55C7JZ62MGB2UXA6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as CLIENT


In [3]:
from anvil.tables import app_tables

answers = app_tables.answers.search()

In [5]:
answers_list = [ {
    "created_at": r["created_at"],
    "question_data": r["question"],
    "question": r["question"]["question"],
    "question_title": r["question"]["title"],
    "question_type": r["question"]["type"],
    "question_level": r["question"]["level"],
    "got_it_right": r["got_it_right"],
    "session":r["session"],
    "user": r["user"]["email"] if r["user"] else ""
    } for r in answers
    ]

answers_list

[{'created_at': datetime.datetime(2024, 8, 15, 16, 44, 24, 571000, tzinfo=<anvil.tz.tzoffset (-3.0 hour offset)>),
  'question_data': <LiveObject: anvil.tables.Row>,
  'question': 'Entropy is a measure of the average uncertainty in a set of outcomes, while cross-entropy measures the difference between two probability distributions.',
  'question_title': 'Probability - LLMs',
  'question_type': 'true_or_false',
  'question_level': 'easy',
  'got_it_right': False,
  'session': '10071df2-2c28-4c47-896d-61ac89d3883a',
  'user': 'teste@teste.com'},
 {'created_at': datetime.datetime(2024, 8, 15, 16, 44, 29, 268000, tzinfo=<anvil.tz.tzoffset (-3.0 hour offset)>),
  'question_data': <LiveObject: anvil.tables.Row>,
  'question': 'Cross-entropy loss measures the difference between predicted probabilities and actual outcomes, while entropy quantifies the uncertainty in a probability distribution.',
  'question_title': 'Probability - LLMs',
  'question_type': 'true_or_false',
  'question_level': '

In [6]:
df = pd.DataFrame.from_dict(answers_list)
df.head()

,created_at,question_data,question,question_title,question_type,question_level,got_it_right,session,user
0,2024-08-15 16:44:24.571000-03:00,"[[explanation, Entropy quantifies the uncertai...",Entropy is a measure of the average uncertaint...,Probability - LLMs,true_or_false,easy,False,10071df2-2c28-4c47-896d-61ac89d3883a,teste@teste.com
1,2024-08-15 16:44:29.268000-03:00,"[[explanation, Cross-entropy loss is used to e...",Cross-entropy loss measures the difference bet...,Probability - LLMs,true_or_false,easy,True,10071df2-2c28-4c47-896d-61ac89d3883a,teste@teste.com
2,2024-08-15 16:44:31.959000-03:00,"[[explanation, Entropy quantifies the uncertai...",Entropy is a measure of uncertainty in a proba...,Probability - LLMs,true_or_false,easy,True,10071df2-2c28-4c47-896d-61ac89d3883a,teste@teste.com
3,2024-08-15 16:44:35.482000-03:00,"[[explanation, Cross-entropy loss is a measure...","Cross-entropy loss is the same as perplexity, ...",Probability - LLMs,true_or_false,easy,True,10071df2-2c28-4c47-896d-61ac89d3883a,teste@teste.com
4,2024-08-15 16:45:10.548000-03:00,"[[explanation, Entropy quantifies the uncertai...",Entropy is a measure of the average uncertaint...,Probability - LLMs,true_or_false,easy,False,a25f3257-1470-41d8-07a6-3a23cf8c1512,teste@teste.com


In [17]:
df.shape

(62, 9)

In [7]:
df_wrong = df.groupby(["question"])["got_it_right"].value_counts().unstack(fill_value=0)
df_wrong = df_wrong.rename(columns={False: 'Wrong', True: 'Right'})
df_wrong = df_wrong.reset_index()  # Reset index to move 'question_question' into a column
df_wrong = df_wrong.sort_values(by="Wrong", ascending=False)
df_wrong = df_wrong[['question', 'Wrong', 'Right']]  # Select the desired columns
df_wrong = df_wrong.reset_index(drop=True)
df_wrong.columns.name = None

df_wrong["Wrong_percent"] = df_wrong["Wrong"] / (df_wrong["Wrong"] + df_wrong["Right"])
df_wrong["wrong_normalized"] = df_wrong["Wrong"] / df_wrong["Wrong"].sum()

df_wrong.head()

df_wrong = df_wrong.set_index('question')

df_wrong.head()

,Wrong,Right,Wrong_percent,wrong_normalized
question,,,,
Precision is the ratio of true positives to the sum of true positives and true negatives.,2,0,1.0,0.250
"Entropy is a measure of the average uncertainty in a set of outcomes, while cross-entropy measures the difference between two probability distributions.",2,0,1.0,0.250
A forged image perturbation of an unauthorized person's face would not make this person be authorized in face identification systems.,1,0,1.0,0.125
Cross-Entropy is always less than or equal to Entropy for a given probability distribution.,1,0,1.0,0.125
Regularization is a technique used to encourage complex models in machine learning.,1,0,1.0,0.125


In [8]:
from datetime import datetime
import pytz

# Assuming 'created_at' is in UTC, convert datetime.now() to UTC
now_aware = datetime.now(pytz.UTC)

df_created = df.groupby(["question"])["created_at"].max().reset_index()
df_created = df_created.set_index("question")
df_created["time_diff_from_now"] = now_aware - df_created["created_at"]
# df_created["time_diff_from_now"] = df_created["time_diff_from_now"].apply( lambda x: int( x.total_seconds() / 60 ) )
df_created["time_diff_from_now"] = df_created["time_diff_from_now"].apply( lambda x: int( x.total_seconds() ) )
df_created = df_created.drop(columns="created_at")
df_created["time_diff_from_now_normalized"] = df_created["time_diff_from_now"] / df_created["time_diff_from_now"].sum()
df_created.head()

,time_diff_from_now,time_diff_from_now_normalized
question,,
A forged image perturbation of an unauthorized person's face would not make this person be authorized in face identification systems.,92891,0.010984
"A linear transformation preserves linear combinations of vectors, but not their magnitudes.",786017,0.092942
Accuracy is a suitable evaluation metric for imbalanced classification problems.,92845,0.010978
Accuracy is the best metric for imbalanced datasets.,94037,0.011119
Bagging and Boosting are types of unsupervised learning techniques.,93996,0.011115


In [9]:
df_answer_stats = df_wrong.join(df_created)
df_answer_stats.head()

,Wrong,Right,Wrong_percent,wrong_normalized,time_diff_from_now,time_diff_from_now_normalized
question,,,,,,
Precision is the ratio of true positives to the sum of true positives and true negatives.,2,0,1.0,0.250,92963,0.010992
"Entropy is a measure of the average uncertainty in a set of outcomes, while cross-entropy measures the difference between two probability distributions.",2,0,1.0,0.250,786911,0.093048
A forged image perturbation of an unauthorized person's face would not make this person be authorized in face identification systems.,1,0,1.0,0.125,92891,0.010984
Cross-Entropy is always less than or equal to Entropy for a given probability distribution.,1,0,1.0,0.125,785992,0.092939
Regularization is a technique used to encourage complex models in machine learning.,1,0,1.0,0.125,94104,0.011127


In [10]:
def score(row):
    weights = {
        "wrong_normalized": 1,
        "time_diff_from_now_normalized": 1
    }
    
    return row["wrong_normalized"] * weights["wrong_normalized"] + row["time_diff_from_now_normalized"] * weights["time_diff_from_now_normalized"]

In [11]:
df_score = df_answer_stats.apply(score, axis=1).sort_values(ascending=False).reset_index()
df_score = df_score.set_index("question")
df_score.columns = ["score"]
df_score

,score
question,
"Entropy is a measure of the average uncertainty in a set of outcomes, while cross-entropy measures the difference between two probability distributions.",0.343048
Precision is the ratio of true positives to the sum of true positives and true negatives.,0.260992
Cross-Entropy is always less than or equal to Entropy for a given probability distribution.,0.217939
Regularization is a technique used to encourage complex models in machine learning.,0.136127
PCA is a supervised learning technique.,0.136125
A forged image perturbation of an unauthorized person's face would not make this person be authorized in face identification systems.,0.135984
"Entropy is a measure of uncertainty in a probability distribution, while cross-entropy measures the difference between two probability distributions.",0.093052
"Cross-entropy loss is the same as perplexity, used to evaluate language models.",0.093052
"Cross-entropy loss measures the difference between predicted probabilities and actual outcomes, while entropy quantifies the uncertainty in a probability distribution.",0.093048


In [12]:
df_data = df[["question_data", "question"]]
df_data = df_data.set_index("question")
df_data.head()

,question_data
question,
"Entropy is a measure of the average uncertainty in a set of outcomes, while cross-entropy measures the difference between two probability distributions.","[[explanation, Entropy quantifies the uncertai..."
"Cross-entropy loss measures the difference between predicted probabilities and actual outcomes, while entropy quantifies the uncertainty in a probability distribution.","[[explanation, Cross-entropy loss is used to e..."
"Entropy is a measure of uncertainty in a probability distribution, while cross-entropy measures the difference between two probability distributions.","[[explanation, Entropy quantifies the uncertai..."
"Cross-entropy loss is the same as perplexity, used to evaluate language models.","[[explanation, Cross-entropy loss is a measure..."
"Entropy is a measure of the average uncertainty in a set of outcomes, while cross-entropy measures the difference between two probability distributions.","[[explanation, Entropy quantifies the uncertai..."


In [13]:
df_data.shape

(62, 1)

In [14]:
df_score.shape

(44, 1)

In [26]:
# df_merged = pd.merge(df_score.reset_index(), df_data.reset_index(), left_index=True, right_index=True, how='left')
df_merged = pd.merge(df_score.reset_index(), df_data.reset_index(), on='question', how='left')
df_merged.shape

(62, 3)

In [27]:
df_merged = df_merged.drop_duplicates()
df_merged

,question,score,question_data
0,Entropy is a measure of the average uncertaint...,0.343048,"[[explanation, Entropy quantifies the uncertai..."
2,Precision is the ratio of true positives to th...,0.260992,"[[explanation, Precision is defined as the rat..."
4,Cross-Entropy is always less than or equal to ...,0.217939,"[[explanation, Cross-Entropy can equal Entropy..."
5,Regularization is a technique used to encourag...,0.136127,"[[explanation, Regularization is used to disco..."
6,PCA is a supervised learning technique.,0.136125,"[[explanation, PCA (Principal Component Analys..."
8,A forged image perturbation of an unauthorized...,0.135984,"[[explanation, A forged image perturbation of ..."
9,Entropy is a measure of uncertainty in a proba...,0.093052,"[[explanation, Entropy quantifies the uncertai..."
10,"Cross-entropy loss is the same as perplexity, ...",0.093052,"[[explanation, Cross-entropy loss is a measure..."
11,Cross-entropy loss measures the difference bet...,0.093048,"[[explanation, Cross-entropy loss is used to e..."
13,A linear transformation preserves linear combi...,0.092942,"[[explanation, Linear transformations maintain..."


In [15]:
# df_data.join(df_score).head()

In [16]:
# df_score_ = df_score.join(df_data)
# # df_score_.head()

In [ ]:
# df_score_